In [2]:
import dill
import yaml
import pickle
import numpy as np
import collections
import string
import copy
from datetime import datetime
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 482 kB 4.2 MB/s 
     |████████████████████████████████| 41 kB 677 kB/s 


In [3]:
from rdflib import Graph, URIRef, Literal, Namespace, XSD
from rdflib.namespace import RDF, RDFS
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#DataSetsfilepath = '/content/drive/MyDrive/sessionNew.pkl'
#  # Save the session
# dill.load_session(DataSetsfilepath)

In [4]:
from IPython.display import clear_output
from os import wait


In [5]:
g = Graph()

g.namespace_manager.bind('ce', Namespace("https://w3id.org/ce#"), override="False")
ce = Namespace("https://w3id.org/ce#")

In [6]:
g.parse('/content/drive/MyDrive/KnowledgeGraph.nt')

<Graph identifier=N89103088e5354e6a8476e5fd09d064e1 (<class 'rdflib.graph.Graph'>)>

In [12]:
# g.serialize(destination='/content/drive/MyDrive/KnowledgeGraph.rdf',format='pretty-xml')

<Graph identifier=N2a64bb6ea788495d9d4867e8545e7cda (<class 'rdflib.graph.Graph'>)>

In [13]:
# g.serialize(destination='/content/drive/MyDrive/KnowledgeGraph.xml',format='xml')

<Graph identifier=N2a64bb6ea788495d9d4867e8545e7cda (<class 'rdflib.graph.Graph'>)>

In [ ]:
# g.parse('/content/drive/MyDrive/KnowledgeGraph.rdf')

# SPARQL CHECKING

In [7]:
!pip install SPARQLWrapper
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as pl
from rdflib import Graph, ConjunctiveGraph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# #data=data.drop_duplicates(subset=['event_id','modified_ts','created_ts',
#                     'name',         'sort_name',            'status',
#                  'website','category',
#                    'ranking_level',  'ranking_in_level','frequency_event' ,
#                 'start_ts',            'end_ts',          'place_id'])

ce = Namespace("https://w3id.org/ce#")
# filepath = '/content/drive/MyDrive/sessionKnowledgeGraph4.pkl'
# dill.load_session(filepath)

In [ ]:

# phone=placesPhoneData.loc[placesPhoneData['place_id']==383]
# phone= phone.iloc[0]
# phone


In [ ]:
# q3 = prepareQuery('''
# SELECT ?b  ?k

#     WHERE {
#       ?event ce:hasPerformance ?performance.
#      ?performance ce:hasDescription ?k.
#      ?k ce:descriptions ?b.
#        }
#        limit 10
#     ''',
#   initNs = { "ce": ce}
# )

Q. Total number of events

In [10]:
# for r in g.query(q3):
#     print(" ",r.k,r.b)
# placesData.columns
q4 = prepareQuery('''
SELECT (count(distinct ?Events) as ?TotalEvents)

    WHERE {
      ?e ce:event_id ?Events.
      
       }
    
    ''',
  initNs = { "ce": ce}
)

In [18]:
list(g.query(q4))[0]

AttributeError: ignored

Q. Total Performances

In [ ]:
q4 = prepareQuery('''
SELECT (count(*) as ?TotalPerformaces)

    WHERE {
      ?e a ce:Schedule;
      ce:hasPerformance ?performaces.
      
       }
    
    ''',
  initNs = { "ce": ce}
)

# eb=tic_desc.loc[tic_desc['event_id']==156630]
# eb
# filepath = '/Users/najmi/Desktop/Final Dissertation/sessionKnowledgeGraph4.pkl'
# dill.load_session(filepath)

In [ ]:
for r in g.query(q4):
    print("ce:hasEvnt ",r.TotalPerformaces)

ce:hasEvnt  354808


In [ ]:
# g.remove((None,ce.hasPerformance,None))

## Schedule Level

### Q. Total Schedules

In [ ]:
q3 = prepareQuery('''
SELECT (count(distinct ?Schedule) as ?TotalSchedules)

    WHERE {
      ?e ce:hasSchedule ?Schedule.
      
       }
    
    ''',
  initNs = { "ce": ce}
)

In [ ]:
for r in g.query(q3):
    print("ce:hasEvnt ",r.TotalSchedules)

ce:hasEvnt  48790


In [ ]:
G = rdflib_to_networkx_multidigraph(g)
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw(G, with_labels=True)

#if not in interactive mode for 
plt.show()

In [ ]:
%matplotlib inline

import sys
from collections import OrderedDict
from rdflib import Graph,URIRef
!pip install gastrodon
from gastrodon import LocalEndpoint,one,QName
import gzip
import pandas as pd
pd.set_option("display.width",100)
pd.set_option("display.max_colwidth",80)

In [ ]:
e=LocalEndpoint(g)
properties=e.select("""
   SELECT ?p (COUNT(*) AS ?cnt) {
      ?s ?p ?o .
   } GROUP BY ?p ORDER BY DESC(?cnt)
""")
properties

,cnt
p,
rdf:type,1576895
ce:type,368300
ce:ts,354808
ce:time_unkown,354808
ce:hasPerformance,354808
...,...
ce:film_metacritic,1
ce:hearing_loop,1
ce:website_company,1


In [ ]:
schedules=e.select("""
   SELECT ?town ?cats (count(?sched) as ?TotalSchedules)
    WHERE {
      {
      ?events a ce:Event;
                ce:hasCategory ?cat;
                ce:hasSchedule ?sched.
      }
     {
      ?sched a ce:Schedule;
               ce:hasPlace ?place.
      ?place   ce:town ?town.
    
      ?cat a ce:Category;
             ce:category ?cats.
    }
     
       }
       GROUP BY ?town ?cats
""")
schedules

TotalSchedules
town         cats                      
Selkirk      Music                   27
Edinburgh    Theatre               3707
             Visual art            1605
             Film                  6074
Innerleithen Days out                22
...                                 ...
Duns         Theatre                  1
Dirleton     Theatre                  1
Whitburn     Days out                 1
Armadale     Comedy                   1
Bathgate     Kids                     1

[731 rows x 1 columns]

In [1]:
schedules.reset_index()


NameError: ignored

In [ ]:
schedules = schedules.reset_index()


In [ ]:
St=schedules[schedules['town']=='St Andrews']
St
Ed=schedules[schedules['town']=='Edinburgh']
Ed

,town,cats,TotalSchedules
1,Edinburgh,Theatre,3707
2,Edinburgh,Visual art,1605
3,Edinburgh,Film,6074
6,Edinburgh,Comedy,5106
9,Edinburgh,Music,7701
12,Edinburgh,Days out,4046
14,Edinburgh,Other,909
15,Edinburgh,Books,3558
17,Edinburgh,Clubs,1522
22,Edinburgh,Kids,830


In [ ]:
import plotly.express as px
def plot_ScheduleFrequency_byCategory_ofCity(city):
  St=properties[properties['town']==city]
  St
  fig=px.bar(St, x='cats', y='TotalSchedules', color='cats',title="Schedule Frequency by category of "+city,labels={'cats': "Category",'TotalSchedules':'Frequency'})
  return fig


In [ ]:
list_Cities={}
a=0
for i in pd.unique(properties['town']):
  list_Cities[i]=plot_ScheduleFrequency_byCategory_ofCity(i)
  

In [ ]:
list_Cities['Edinburgh'].show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def plot_comparison(City_A,City_B,properties):
  City_1=properties[properties['town']==City_A]
  City_1
  City_2=properties[properties['town']==City_B]
  City_2
  fig = go.Figure()
  fig.add_trace(go.Bar(x=City_1["cats"], y=City_1["TotalSchedules"], name=City_1['town'].iloc[0],
        hovertemplate="City: %s<br>Event Type: %%{x}<br>Schedules = %%{y}<extra></extra>"% City_1['town'].iloc[0]))
  fig.add_trace(go.Bar(x=City_2["cats"], y=City_2["TotalSchedules"], name=City_2['town'].iloc[0],
        hovertemplate="City: %s<br>Event Type: %%{x}<br>Schedules = %%{y}<extra></extra>"% City_2['town'].iloc[0]))
  return fig



In [ ]:
pd.unique(properties['town'])
a='Edinburgh'
b='St Andrews'
plot_comparison(a,b,properties)

## Q. Total Schedules of a Category x in a City y


In [ ]:
q8 = prepareQuery('''
SELECT (count(?sched) as ?totalCityEvents)
    WHERE {
      {
      ?events a ce:Event;
                ce:hasCategory ?cat;
                ce:hasSchedule ?sched.
      }
     {
      ?sched a ce:Schedule;
               ce:hasPlace ?place.
      ?place   ce:town ?town.
    
      ?cat a ce:Category;
             ce:category ?cats.
    }
     
       }
       
       
       
   
    ''',
  initNs = { "ce": ce}
)


# eb=tic_desc.loc[tic_desc['event_id']==156630]
# eb
# filepath = '/Users/najmi/Desktop/Final Dissertation/sessionKnowledgeGraph4.pkl'
# dill.load_session(filepath)

In [ ]:
for r in g.query(q8,initBindings = {'?cats' : Literal('Music', datatype=XSD.string),'?town' : Literal('Edinburgh', datatype=XSD.string)}):
  print(r.totalCityEvents)
  print(r)

In [ ]:
df

In [ ]:
q8 = prepareQuery('''
SELECT (count(?sched) as ?totalCityEvents) 

    WHERE {
      {?events a ce:Event;
      ce:hasCategory ?cat;
      ce:hasSchedule ?sched.}
     { 
       ?sched a ce:Schedule;
        ce:hasPlace ?place.
      ?place ce:town ?town.
      ?cat a ce:Category;
    ce:category ?cats.}
      
       }
       
       
       
       
    
    ''',
  initNs = { "ce": ce}
)

In [ ]:
for r in g.query(q8,initBindings = {'?cats' : Literal('Music', datatype=XSD.string),'?town' : Literal('St Andrews', datatype=XSD.string)}):
  print(r.totalCityEvents)

In [ ]:
q8 = prepareQuery('''
SELECT ?lat ?long(count(?sched) as ?totalCityEvents) 

    WHERE {
      {?events a ce:Event;
      ce:hasSchedule ?sched.}
     { 
       ?sched a ce:Schedule;
        ce:hasPlace ?place.
      ?place ce:town ?town.
      ?place ce:hasLocation ?loc.
      ?loc a ce:Loc;
      ce:latitude ?lat;
      ce:longitude ?long}
      
       }
       
       
       
       
       
       
    
    ''',
  initNs = { "ce": ce}
)

In [ ]:
for r in g.query(q8,initBindings = {'?town' : Literal('St Andrews', datatype=XSD.string)}):
  print(r.totalCityEvents)

KeyboardInterrupt: ignored

In [ ]:
schedules=e.select('''
SELECT ?town (count(?sched) as ?totalCityEvents) 

    WHERE {
      {?events a ce:Event;
      ce:hasSchedule ?sched.}
     { 
       ?sched a ce:Schedule;
        ce:hasPlace ?place.
      ?place ce:town ?town.
      }
      
      
       }
       Group by ?town
       
       
       
       
       
       
    
    ''')
schedules

,totalCityEvents
town,
Edinburgh,36962
Kirkcaldy,885
Crail,161
Glenrothes,443
,59
...,...
By Collessie,1
St. Monans,1
East Lothian,2


In [ ]:
schedules=schedules.reset_index()

In [ ]:
px.bar(schedules,x='town',y='totalCityEvents',color='town',labels={'town': "Cities", 'totalCityEvents': 'Total Events'})

NameError: ignored

In [ ]:
locations=e.select('''
SELECT ?town  ?lat ?long ?id (count (?schedules )as ?frequency) 

    WHERE {
      {

       ?events ce:event_id ?id;
       ce:hasSchedule ?schedules.
       ?schedules a ce:Schedule;
       ce:hasPlace ?place.
       }
       
  {
    ?place a ce:Place;
        ce:town ?town;
        ce:hasLocation ?loc.
      ?loc  ce:latitude ?lat;
        ce:longitude ?long.
        

  }    
      
      
       }
       group by ?lat ?long ?id
       order by desc (?frequency)
       
       
       
       
       
       
       
    
    ''')
locations

,,,town,frequency
lat,long,id,,
55.949927,-3.193409,335212,Edinburgh,9
55.950398,-3.186481,197665,Edinburgh,9
55.948200,-3.191460,245723,Edinburgh,8
55.964443,-3.174362,212897,Edinburgh,8
55.940678,-3.368805,569635,Edinburgh,8
...,...,...,...,...
55.892400,-3.519140,580098,Livingston,1
55.938522,-3.232600,1523882,Edinburgh,1
55.946400,-3.206000,418867,Edinburgh,1


In [ ]:
locations=locations.reset_index()
EdMapLoc=locations[locations['town']=='Edinburgh']

In [ ]:
EdMapLoc

,lat,long,id,town,frequency
0,55.949927,-3.193409,335212,Edinburgh,9
1,55.950398,-3.186481,197665,Edinburgh,9
2,55.964443,-3.174362,212897,Edinburgh,8
3,55.948200,-3.191460,245723,Edinburgh,8
4,55.940678,-3.368805,569635,Edinburgh,8
...,...,...,...,...,...
45407,55.941978,-3.148544,1399904,Edinburgh,1
45408,55.948200,-3.191460,364026,Edinburgh,1
45409,55.946642,-3.192276,1341145,Edinburgh,1
45410,55.954070,-3.195862,1272727,Edinburgh,1


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
fig1 = px.scatter_mapbox(EdMapLoc, lat="lat", lon="long", hover_name="town", hover_data=["id", "frequency"],
                        color_discrete_sequence=["red"], zoom=3, height=300)
fig1.update_layout(mapbox_style="open-street-map")
fig1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## Q. Frequency of schedules of a category x in a town y in a year z

In [ ]:
# requency of number of events/schedules/performances per city/town and/or date and/or category
# Histogram of events/schedules/performances per city/town and/or category
# Map of events/schedules/performances per city/town and/or date and/or category
# Histogram of performances sold-out and/or cancelled per city/town and/or category
q8 = prepareQuery('''
SELECT (count(?sched) as ?totalCityEvents) 

    WHERE {
      {?events a ce:Event;
      ce:hasCategory ?cat;
      ce:hasSchedule ?sched.}
     { 
       ?sched a ce:Schedule;
        ce:start_ts ?date;
        ce:hasPlace ?place.
      ?place ce:town ?town.
      ?cat a ce:Category;
    ce:category ?cats.}
    FILTER (?date > "2019-01-23T10:20:13+05:30"^^xsd:dateTime)
       FILTER (?date < "2022-05-23T10:20:13+05:30"^^xsd:dateTime)
      
       }
       
       
       
       
    
    ''',
  initNs = { "ce": ce}
)


In [ ]:
for r in g.query(q8,initBindings = {'?cats' : Literal('Music', datatype=XSD.string),'?town' : Literal('St Andrews', datatype=XSD.string)}):
  print(r.totalCityEvents)

## Q. Total Schedules with a particular tag and category in a city

In [ ]:
q8 = prepareQuery('''
SELECT (count(distinct ?sched) as ?totalCityEvents) 

    WHERE {
      {?events a ce:Event;
      ce:hasCategory ?cat;
      ce:hasTag ?tags;
      ce:hasSchedule ?sched.
      }
     { 
       ?sched a ce:Schedule;
        ce:hasPlace ?place.
      ?place ce:town ?town.
      ?cat a ce:Category;
    ce:category ?cats.
    ?tags a ce:Tag;
    ce:tags ?tag.
    }
      
       }
       
       
       
    
    ''',
  initNs = { "ce": ce}
)

In [ ]:
for r in g.query(q8,initBindings ={'?tag' : Literal('Music', datatype=XSD.string),'?cats' : Literal('Music', datatype=XSD.string),'?town' : Literal('Edinburgh', datatype=XSD.string)}):
  print(r.totalCityEvents)

In [ ]:
print(len(g))

## Event Level

In [ ]:
q7 = prepareQuery('''
SELECT (count(distinct ?scheds) as ?totalCityEvents) 

    WHERE {
      {?events ce:hasSchedules ?scheds;
      ce:hasCategory ?cat.}
     {?cat ce:category ?cats.} 
     {?scheds ce:hasPlace ?place.
     ?place ce:town ?town.} 
    


      

       }
       
       
    
    ''',
  initNs = { "ce": ce}
)


In [ ]:
for r in g.query(q7,initBindings = {'?cats' : Literal('Music', datatype=XSD.string),'?town' : Literal('St Andrews', datatype=XSD.string)}):
  print(r.totalCityEvents)